# Feature engineering for the simulated dataset

In [1]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = "{:.1f}".format

In [2]:
from data.load_sim_data import load_all_sim_data
transactions_df, customers_df, terminals_df = load_all_sim_data()

In [3]:
small_dataset = False
if small_dataset:
    START_DATE = "2018-04-01"
    END_DATE = "2018-04-30"
    transactions_df = transactions_df[transactions_df['TX_DATETIME'].between(START_DATE, END_DATE, inclusive='both')]

In [4]:
transactions_df.shape

(1754155, 10)

## Baseline features
The following features will be used to build the baseline model. With each model iteration more complex features can be added. But these baseline features can be deemed as the bare minimum.

#### Weekend and weekday

In [5]:
def is_weekend(tx_datetime_series):
    dow_is_weekend = tx_datetime_series.weekday() >= 5 # 0 Monday, 6 Sunday
    return int(dow_is_weekend)

In [6]:
transactions_df['TX_ON_WEEKEND'] = transactions_df.TX_DATETIME.apply(is_weekend)

In [7]:
# quick checks
# 1st April is a Sunday
# 2nd April is a Monday
# 14th April is a Saturday
def check_weekend_flag(transactions_df, check_dates={'2018-04-01':1, '2018-04-02':0, '2018-04-14': 1}):
    for check_date, expected_flag in check_dates.items():
        flag_val = transactions_df[transactions_df['TX_DATE'] == check_date]['TX_ON_WEEKEND'].unique()
        assert flag_val == expected_flag, "Weekend flag not correct for: " + str(check_date) + ". Expected " + str(expected_flag) + " but got " + str(flag_val)
    return None

In [8]:
check_weekend_flag(transactions_df, check_dates={'2018-04-01':1, '2018-04-02':0, '2018-04-14': 1})
transactions_df[['TX_DATE', 'TX_ON_WEEKEND']].groupby('TX_DATE').max('').head(10)

,TX_ON_WEEKEND
TX_DATE,
2018-04-01,1
2018-04-02,0
2018-04-03,0
2018-04-04,0
2018-04-05,0
2018-04-06,0
2018-04-07,1
2018-04-08,1
2018-04-09,0


#### At night or day

In [9]:
def is_night(tx_datetime_series):
    # between 12am and 6am we consider night
    txn_at_night = tx_datetime_series.hour < 6
    return int(txn_at_night)

In [10]:
transactions_df['TX_AT_NIGHT'] = transactions_df.TX_DATETIME.apply(is_night)

In [11]:
# Quick checks
transactions_df.sample(3, random_state=12)[['TX_DATETIME','TX_ON_WEEKEND', 'TX_AT_NIGHT']]

,TX_DATETIME,TX_ON_WEEKEND,TX_AT_NIGHT
1084909,2018-07-23 06:24:22,0,0
630587,2018-06-05 15:34:21,0,0
251908,2018-04-27 08:25:23,0,0


### Customer features
These features will be RFM - Recency, Frequency and Monetary value. These types of features are also commonly used in retail customer analytics.

In [12]:
# Start by building for 1 customer
customer_txns = transactions_df[transactions_df['CUSTOMER_ID'] == 0]
customer_txns.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DATE,TX_ON_WEEKEND,TX_AT_NIGHT
1758,1758,2018-04-01 07:19:05,0,6076,123.6,26345,0,0,0,2018-04-01,1,0
8275,8275,2018-04-01 18:00:16,0,858,77.3,64816,0,0,0,2018-04-01,1,0
8640,8640,2018-04-01 19:02:02,0,6698,46.5,68522,0,0,0,2018-04-01,1,0
12169,12169,2018-04-02 08:51:06,0,6569,54.7,118266,1,0,0,2018-04-02,0,0
15764,15764,2018-04-02 14:05:38,0,7707,63.3,137138,1,0,0,2018-04-02,0,0


In [13]:
def generate_customer_rfm(customer_txns, window_sizes=(1, 7, 30)):
    customer_txns = customer_txns.sort_values("TX_DATETIME")
    customer_txns.index = customer_txns.TX_DATETIME # allows us to use rolling functions

    # Window features such as transaction sum amount, average amount and count
    for window in window_sizes:
        SUB_TX_AMOUNT = customer_txns['TX_AMOUNT'].rolling(str(window) + 'd').sum()
        SUB_TX_COUNT = customer_txns['TX_AMOUNT'].rolling(str(window) + 'd').count()
        SUB_TX_AVG_AMOUNT = SUB_TX_AMOUNT/SUB_TX_COUNT # note that average at start of the data is missing past days so would only have a few days of data.

        customer_txns['CUSTOMER_ID_' + str(window) + '_DAY_WINDOW_SUM_AMOUNT'] = SUB_TX_AMOUNT
        customer_txns['CUSTOMER_ID_' + str(window) + '_DAY_WINDOW_COUNT'] = SUB_TX_COUNT
        customer_txns['CUSTOMER_ID_' + str(window) + '_DAY_WINDOW_AVG_AMOUNT'] = SUB_TX_AVG_AMOUNT

    # Recency feature
    customer_txns['CUSTOMER_ID_DAYS_SINCE_LAST_TXN'] = customer_txns['TX_DATETIME'].diff().dt.days.fillna(0).astype(int)

    # drop index according
    customer_txns.reset_index(drop=True, inplace=True)

    return customer_txns

In [14]:
customer_txns_with_rfm = generate_customer_rfm(customer_txns)
customer_txns_with_rfm.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DATE,...,CUSTOMER_ID_1_DAY_WINDOW_SUM_AMOUNT,CUSTOMER_ID_1_DAY_WINDOW_COUNT,CUSTOMER_ID_1_DAY_WINDOW_AVG_AMOUNT,CUSTOMER_ID_7_DAY_WINDOW_SUM_AMOUNT,CUSTOMER_ID_7_DAY_WINDOW_COUNT,CUSTOMER_ID_7_DAY_WINDOW_AVG_AMOUNT,CUSTOMER_ID_30_DAY_WINDOW_SUM_AMOUNT,CUSTOMER_ID_30_DAY_WINDOW_COUNT,CUSTOMER_ID_30_DAY_WINDOW_AVG_AMOUNT,CUSTOMER_ID_DAYS_SINCE_LAST_TXN
0,1758,2018-04-01 07:19:05,0,6076,123.6,26345,0,0,0,2018-04-01,...,123.6,1.0,123.6,123.6,1.0,123.6,123.6,1.0,123.6,0
1,8275,2018-04-01 18:00:16,0,858,77.3,64816,0,0,0,2018-04-01,...,200.9,2.0,100.5,200.9,2.0,100.5,200.9,2.0,100.5,0
2,8640,2018-04-01 19:02:02,0,6698,46.5,68522,0,0,0,2018-04-01,...,247.4,3.0,82.5,247.4,3.0,82.5,247.4,3.0,82.5,0
3,12169,2018-04-02 08:51:06,0,6569,54.7,118266,1,0,0,2018-04-02,...,178.6,3.0,59.5,302.2,4.0,75.5,302.2,4.0,75.5,0
4,15764,2018-04-02 14:05:38,0,7707,63.3,137138,1,0,0,2018-04-02,...,241.9,4.0,60.5,365.5,5.0,73.1,365.5,5.0,73.1,0


In [15]:
transactions_df = transactions_df.groupby('CUSTOMER_ID').apply(generate_customer_rfm)

In [16]:
transactions_df = transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)
transactions_df.index = transactions_df['TRANSACTION_ID']

In [17]:
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DATE,...,CUSTOMER_ID_1_DAY_WINDOW_SUM_AMOUNT,CUSTOMER_ID_1_DAY_WINDOW_COUNT,CUSTOMER_ID_1_DAY_WINDOW_AVG_AMOUNT,CUSTOMER_ID_7_DAY_WINDOW_SUM_AMOUNT,CUSTOMER_ID_7_DAY_WINDOW_COUNT,CUSTOMER_ID_7_DAY_WINDOW_AVG_AMOUNT,CUSTOMER_ID_30_DAY_WINDOW_SUM_AMOUNT,CUSTOMER_ID_30_DAY_WINDOW_COUNT,CUSTOMER_ID_30_DAY_WINDOW_AVG_AMOUNT,CUSTOMER_ID_DAYS_SINCE_LAST_TXN
TRANSACTION_ID,,,,,,,,,,,,,,,,,,,,,
0,0,2018-04-01 00:00:31,596,3156,57.2,31,0,0,0,2018-04-01,...,57.2,1.0,57.2,57.2,1.0,57.2,57.2,1.0,57.2,0
1,1,2018-04-01 00:02:10,4961,3412,81.5,130,0,0,0,2018-04-01,...,81.5,1.0,81.5,81.5,1.0,81.5,81.5,1.0,81.5,0
2,2,2018-04-01 00:07:56,2,1365,146.0,476,0,0,0,2018-04-01,...,146.0,1.0,146.0,146.0,1.0,146.0,146.0,1.0,146.0,0
3,3,2018-04-01 00:09:29,4128,8737,64.5,569,0,0,0,2018-04-01,...,64.5,1.0,64.5,64.5,1.0,64.5,64.5,1.0,64.5,0
4,4,2018-04-01 00:10:34,927,9906,51.0,634,0,0,0,2018-04-01,...,51.0,1.0,51.0,51.0,1.0,51.0,51.0,1.0,51.0,0


In [18]:
# Quick check - using human eyes, and stopping short of recalculating the cols
random_cust_id = transactions_df['CUSTOMER_ID'].sample(1, random_state=11).values[0]
random_cust_id

2188

In [19]:
transactions_df[transactions_df['CUSTOMER_ID'] == random_cust_id][['TX_DATE', 'CUSTOMER_ID_1_DAY_WINDOW_COUNT', 'CUSTOMER_ID_1_DAY_WINDOW_SUM_AMOUNT', 'CUSTOMER_ID_1_DAY_WINDOW_AVG_AMOUNT']]

,TX_DATE,CUSTOMER_ID_1_DAY_WINDOW_COUNT,CUSTOMER_ID_1_DAY_WINDOW_SUM_AMOUNT,CUSTOMER_ID_1_DAY_WINDOW_AVG_AMOUNT
TRANSACTION_ID,,,,
2454,2018-04-01,1.0,102.4,102.4
10021,2018-04-02,2.0,242.3,121.1
15922,2018-04-02,2.0,228.8,114.4
16561,2018-04-02,3.0,309.4,103.1
30637,2018-04-04,1.0,26.7,26.7
...,...,...,...,...
1743557,2018-09-29,4.0,318.1,79.5
1743699,2018-09-29,5.0,404.0,80.8
1745322,2018-09-30,6.0,471.8,78.6


In [20]:
transactions_df[transactions_df['CUSTOMER_ID'] == random_cust_id][['TX_DATE', 'CUSTOMER_ID_7_DAY_WINDOW_COUNT', 'CUSTOMER_ID_7_DAY_WINDOW_SUM_AMOUNT', 'CUSTOMER_ID_7_DAY_WINDOW_AVG_AMOUNT']]

,TX_DATE,CUSTOMER_ID_7_DAY_WINDOW_COUNT,CUSTOMER_ID_7_DAY_WINDOW_SUM_AMOUNT,CUSTOMER_ID_7_DAY_WINDOW_AVG_AMOUNT
TRANSACTION_ID,,,,
2454,2018-04-01,1.0,102.4,102.4
10021,2018-04-02,2.0,242.3,121.1
15922,2018-04-02,3.0,331.2,110.4
16561,2018-04-02,4.0,411.8,103.0
30637,2018-04-04,5.0,438.5,87.7
...,...,...,...,...
1743557,2018-09-29,15.0,1214.7,81.0
1743699,2018-09-29,16.0,1300.6,81.3
1745322,2018-09-30,17.0,1368.3,80.5


In [21]:
transactions_df[transactions_df['CUSTOMER_ID'] == random_cust_id][['TX_DATE', 'CUSTOMER_ID_30_DAY_WINDOW_COUNT', 'CUSTOMER_ID_30_DAY_WINDOW_SUM_AMOUNT', 'CUSTOMER_ID_30_DAY_WINDOW_AVG_AMOUNT']]

,TX_DATE,CUSTOMER_ID_30_DAY_WINDOW_COUNT,CUSTOMER_ID_30_DAY_WINDOW_SUM_AMOUNT,CUSTOMER_ID_30_DAY_WINDOW_AVG_AMOUNT
TRANSACTION_ID,,,,
2454,2018-04-01,1.0,102.4,102.4
10021,2018-04-02,2.0,242.3,121.1
15922,2018-04-02,3.0,331.2,110.4
16561,2018-04-02,4.0,411.8,103.0
30637,2018-04-04,5.0,438.5,87.7
...,...,...,...,...
1743557,2018-09-29,73.0,6901.0,94.5
1743699,2018-09-29,74.0,6986.9,94.4
1745322,2018-09-30,74.0,6960.6,94.1


In [22]:
transactions_df[transactions_df['CUSTOMER_ID'] == random_cust_id][['TX_DATETIME', 'CUSTOMER_ID_DAYS_SINCE_LAST_TXN']]

,TX_DATETIME,CUSTOMER_ID_DAYS_SINCE_LAST_TXN
TRANSACTION_ID,,
2454,2018-04-01 08:34:18,0
10021,2018-04-02 03:47:02,0
15922,2018-04-02 14:21:00,0
16561,2018-04-02 15:22:39,0
30637,2018-04-04 07:16:33,1
...,...,...
1743557,2018-09-29 18:48:15,0
1743699,2018-09-29 19:15:51,0
1745322,2018-09-30 04:52:26,0


### Terminal features

Determine a risk score for each terminal.

We will include a delay period in this since the `TX_FRAUD` flag will likely not be known immediately. This will usually change once a customer has reported fraud for example.
Therefore, we will set a delay period of 1w assuming that fraud will be reported within a week.

`Risk = num of fraud txns / num of total txns, for a given period`.

In [23]:
transactions_df.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DATE,...,CUSTOMER_ID_1_DAY_WINDOW_SUM_AMOUNT,CUSTOMER_ID_1_DAY_WINDOW_COUNT,CUSTOMER_ID_1_DAY_WINDOW_AVG_AMOUNT,CUSTOMER_ID_7_DAY_WINDOW_SUM_AMOUNT,CUSTOMER_ID_7_DAY_WINDOW_COUNT,CUSTOMER_ID_7_DAY_WINDOW_AVG_AMOUNT,CUSTOMER_ID_30_DAY_WINDOW_SUM_AMOUNT,CUSTOMER_ID_30_DAY_WINDOW_COUNT,CUSTOMER_ID_30_DAY_WINDOW_AVG_AMOUNT,CUSTOMER_ID_DAYS_SINCE_LAST_TXN
TRANSACTION_ID,,,,,,,,,,,,,,,,,,,,,
0,0,2018-04-01 00:00:31,596,3156,57.2,31,0,0,0,2018-04-01,...,57.2,1.0,57.2,57.2,1.0,57.2,57.2,1.0,57.2,0
1,1,2018-04-01 00:02:10,4961,3412,81.5,130,0,0,0,2018-04-01,...,81.5,1.0,81.5,81.5,1.0,81.5,81.5,1.0,81.5,0
2,2,2018-04-01 00:07:56,2,1365,146.0,476,0,0,0,2018-04-01,...,146.0,1.0,146.0,146.0,1.0,146.0,146.0,1.0,146.0,0
3,3,2018-04-01 00:09:29,4128,8737,64.5,569,0,0,0,2018-04-01,...,64.5,1.0,64.5,64.5,1.0,64.5,64.5,1.0,64.5,0
4,4,2018-04-01 00:10:34,927,9906,51.0,634,0,0,0,2018-04-01,...,51.0,1.0,51.0,51.0,1.0,51.0,51.0,1.0,51.0,0


In [24]:
def generate_terminal_risk(terminal_txns, time_delay=7, window_sizes=(1, 7, 30)):

    terminal_txns.index = terminal_txns['TX_DATETIME']
    DELAY_FRAUD_TXN_COUNT = terminal_txns['TX_FRAUD'].rolling(str(time_delay) + 'd').sum('TX_FRAUD')
    DELAY_ALL_TXN_COUNT = terminal_txns['TX_FRAUD'].rolling(str(time_delay) + 'd').sum('count')

    for window in window_sizes:
        FRAUD_TXN_COUNT_NODELAY = terminal_txns['TX_FRAUD'].rolling(str(time_delay + window) + 'd').sum('TX_FRAUD')
        ALL_TXN_COUNT_NODELAY = terminal_txns['TX_FRAUD'].rolling(str(time_delay + window) + 'd').count()

        FRAUD_TXN_COUNT = FRAUD_TXN_COUNT_NODELAY - DELAY_FRAUD_TXN_COUNT
        ALL_TXN_COUNT = ALL_TXN_COUNT_NODELAY - DELAY_ALL_TXN_COUNT

        RISK_PERC = FRAUD_TXN_COUNT / ALL_TXN_COUNT

        terminal_txns['TERMINAL_ID_' + 'FRAUD_RISK_' + str(window) + '_DAY_WINDOW'] = RISK_PERC
        terminal_txns['TERMINAL_ID_' + 'FRAUD_TXN_COUNT_' + str(window) + '_DAY_WINDOW'] = FRAUD_TXN_COUNT
        terminal_txns['TERMINAL_ID_' + 'TXN_COUNT_' + str(window) + '_DAY_WINDOW'] = ALL_TXN_COUNT

    terminal_txns.fillna(0, inplace=True)
    terminal_txns.reset_index(drop=True, inplace=True)

    return terminal_txns

In [25]:
terminal_txns = transactions_df[transactions_df['TERMINAL_ID'] == 0].copy()
generate_terminal_risk(terminal_txns)

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DATE,...,CUSTOMER_ID_DAYS_SINCE_LAST_TXN,TERMINAL_ID_FRAUD_RISK_1_DAY_WINDOW,TERMINAL_ID_FRAUD_TXN_COUNT_1_DAY_WINDOW,TERMINAL_ID_TXN_COUNT_1_DAY_WINDOW,TERMINAL_ID_FRAUD_RISK_7_DAY_WINDOW,TERMINAL_ID_FRAUD_TXN_COUNT_7_DAY_WINDOW,TERMINAL_ID_TXN_COUNT_7_DAY_WINDOW,TERMINAL_ID_FRAUD_RISK_30_DAY_WINDOW,TERMINAL_ID_FRAUD_TXN_COUNT_30_DAY_WINDOW,TERMINAL_ID_TXN_COUNT_30_DAY_WINDOW
0,9579,2018-04-02 01:00:01,3440,0,16.1,90001,1,0,0,2018-04-02,...,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,12806,2018-04-02 09:49:55,3302,0,67.8,121795,1,0,0,2018-04-02,...,0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,2.0
2,24184,2018-04-03 12:14:41,3790,0,26.8,216881,2,0,0,2018-04-03,...,0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,3.0
3,46284,2018-04-05 16:47:41,1125,0,40.5,406061,4,0,0,2018-04-05,...,0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0
4,58807,2018-04-07 06:05:21,1125,0,48.4,540321,6,0,0,2018-04-07,...,0,0.0,0.0,5.0,0.0,0.0,5.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,1731401,2018-09-28 13:41:40,2499,0,4.7,15601300,180,0,0,2018-09-28,...,1,0.0,0.0,11.0,0.0,0.0,19.0,0.0,0.0,48.0
174,1734895,2018-09-28 22:05:28,3302,0,50.8,15631528,180,0,0,2018-09-28,...,0,0.0,0.0,11.0,0.0,0.0,19.0,0.0,0.0,48.0
175,1736214,2018-09-29 05:49:27,1408,0,104.0,15659367,181,0,0,2018-09-29,...,0,0.0,0.0,12.0,0.0,0.0,19.0,0.0,0.0,49.0
176,1739269,2018-09-29 11:12:46,1408,0,38.8,15678766,181,0,0,2018-09-29,...,0,0.0,0.0,13.0,0.0,0.0,20.0,0.0,0.0,50.0


In [26]:
transactions_df = transactions_df.groupby('TERMINAL_ID').apply(generate_terminal_risk)

In [27]:
transactions_df = transactions_df.sort_values('TX_DATETIME').reset_index(drop=True)
transactions_df.index = transactions_df['TRANSACTION_ID']

In [28]:
transactions_df.iloc[transactions_df['TERMINAL_ID_FRAUD_RISK_30_DAY_WINDOW'].idxmax()]

TRANSACTION_ID                                             96159
TX_DATETIME                                  2018-04-11 03:02:54
CUSTOMER_ID                                                 3493
TERMINAL_ID                                                  898
TX_AMOUNT                                                   25.0
TX_TIME_SECONDS                                           874974
TX_TIME_DAYS                                                  10
TX_FRAUD                                                       1
TX_FRAUD_SCENARIO                                              2
TX_DATE                                      2018-04-11 00:00:00
TX_ON_WEEKEND                                                  0
TX_AT_NIGHT                                                    1
CUSTOMER_ID_1_DAY_WINDOW_SUM_AMOUNT                         53.0
CUSTOMER_ID_1_DAY_WINDOW_COUNT                               4.0
CUSTOMER_ID_1_DAY_WINDOW_AVG_AMOUNT                         13.3
CUSTOMER_ID_7_DAY_WINDOW_

In [29]:
# Quick check - using human eyes, and stopping short of recalculating the cols
random_terminal_id = transactions_df[transactions_df['TERMINAL_ID_FRAUD_RISK_30_DAY_WINDOW'] > 0]['TERMINAL_ID'].sample(1, random_state=11).values[0]
random_terminal_id

9488

In [30]:
transactions_df[(transactions_df['TERMINAL_ID'] == random_terminal_id) & (transactions_df['TX_FRAUD'] == 1)]
# 4th April had a fraud txn of $103

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT,TX_TIME_SECONDS,TX_TIME_DAYS,TX_FRAUD,TX_FRAUD_SCENARIO,TX_DATE,...,CUSTOMER_ID_DAYS_SINCE_LAST_TXN,TERMINAL_ID_FRAUD_RISK_1_DAY_WINDOW,TERMINAL_ID_FRAUD_TXN_COUNT_1_DAY_WINDOW,TERMINAL_ID_TXN_COUNT_1_DAY_WINDOW,TERMINAL_ID_FRAUD_RISK_7_DAY_WINDOW,TERMINAL_ID_FRAUD_TXN_COUNT_7_DAY_WINDOW,TERMINAL_ID_TXN_COUNT_7_DAY_WINDOW,TERMINAL_ID_FRAUD_RISK_30_DAY_WINDOW,TERMINAL_ID_FRAUD_TXN_COUNT_30_DAY_WINDOW,TERMINAL_ID_TXN_COUNT_30_DAY_WINDOW
TRANSACTION_ID,,,,,,,,,,,,,,,,,,,,,
99250,99250,2018-04-11 09:58:26,245,9488,384.5,899906,10,1,3,2018-04-11,...,0,0.0,0.0,6.0,0.0,0.0,6.0,0.0,0.0,6.0
1124358,1124358,2018-07-27 08:42:48,1834,9488,66.9,10140168,117,1,2,2018-07-27,...,0,0.0,0.0,8.0,0.0,0.0,11.0,0.0,0.0,37.0
1129117,1129117,2018-07-27 15:58:01,2558,9488,110.6,10166281,117,1,2,2018-07-27,...,0,0.0,0.0,6.0,0.0,0.0,11.0,0.0,0.0,37.0
1187972,1187972,2018-08-02 18:50:48,600,9488,5.7,10695048,123,1,2,2018-08-02,...,0,0.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,33.0
1200081,1200081,2018-08-04 07:00:53,1156,9488,68.1,10825253,125,1,2,2018-08-04,...,2,1.0,2.0,2.0,0.3,2.0,7.0,0.1,2.0,35.0
1205893,1205893,2018-08-04 16:03:25,3435,9488,25.6,10857805,125,1,2,2018-08-04,...,1,0.0,0.0,0.0,0.3,2.0,7.0,0.1,2.0,35.0
1206270,1206270,2018-08-04 16:43:55,671,9488,35.8,10860235,125,1,2,2018-08-04,...,0,0.0,0.0,0.0,0.3,2.0,7.0,0.1,2.0,35.0
1233464,1233464,2018-08-07 14:19:21,3435,9488,36.8,11110761,128,1,2,2018-08-07,...,1,0.0,0.0,0.0,0.4,2.0,5.0,0.1,2.0,30.0
1244654,1244654,2018-08-08 16:50:25,671,9488,17.9,11206225,129,1,2,2018-08-08,...,0,0.0,0.0,0.0,0.5,2.0,4.0,0.1,2.0,29.0


In [31]:
transactions_df[transactions_df['TERMINAL_ID'] == random_terminal_id][['TX_DATETIME', 'TERMINAL_ID_FRAUD_TXN_COUNT_1_DAY_WINDOW', 'TERMINAL_ID_TXN_COUNT_1_DAY_WINDOW', 'TERMINAL_ID_FRAUD_RISK_1_DAY_WINDOW']]

,TX_DATETIME,TERMINAL_ID_FRAUD_TXN_COUNT_1_DAY_WINDOW,TERMINAL_ID_TXN_COUNT_1_DAY_WINDOW,TERMINAL_ID_FRAUD_RISK_1_DAY_WINDOW
TRANSACTION_ID,,,,
30686,2018-04-04 07:23:17,0.0,1.0,0.0
34395,2018-04-04 13:04:40,0.0,2.0,0.0
41647,2018-04-05 09:41:47,0.0,3.0,0.0
42835,2018-04-05 11:24:38,0.0,4.0,0.0
43224,2018-04-05 11:57:24,0.0,5.0,0.0
...,...,...,...,...
1725313,2018-09-27 22:05:06,0.0,12.0,0.0
1732564,2018-09-28 15:32:22,0.0,13.0,0.0
1735365,2018-09-29 02:35:48,0.0,14.0,0.0


In [32]:
transactions_df[transactions_df['TERMINAL_ID'] == random_terminal_id][['TX_DATETIME', 'TERMINAL_ID_FRAUD_TXN_COUNT_7_DAY_WINDOW', 'TERMINAL_ID_TXN_COUNT_7_DAY_WINDOW', 'TERMINAL_ID_FRAUD_RISK_7_DAY_WINDOW']]

,TX_DATETIME,TERMINAL_ID_FRAUD_TXN_COUNT_7_DAY_WINDOW,TERMINAL_ID_TXN_COUNT_7_DAY_WINDOW,TERMINAL_ID_FRAUD_RISK_7_DAY_WINDOW
TRANSACTION_ID,,,,
30686,2018-04-04 07:23:17,0.0,1.0,0.0
34395,2018-04-04 13:04:40,0.0,2.0,0.0
41647,2018-04-05 09:41:47,0.0,3.0,0.0
42835,2018-04-05 11:24:38,0.0,4.0,0.0
43224,2018-04-05 11:57:24,0.0,5.0,0.0
...,...,...,...,...
1725313,2018-09-27 22:05:06,0.0,17.0,0.0
1732564,2018-09-28 15:32:22,0.0,15.0,0.0
1735365,2018-09-29 02:35:48,0.0,16.0,0.0


In [33]:
transactions_df[transactions_df['TERMINAL_ID'] == random_terminal_id][['TX_DATETIME', 'TERMINAL_ID_FRAUD_TXN_COUNT_30_DAY_WINDOW', 'TERMINAL_ID_TXN_COUNT_30_DAY_WINDOW', 'TERMINAL_ID_FRAUD_RISK_30_DAY_WINDOW']]

,TX_DATETIME,TERMINAL_ID_FRAUD_TXN_COUNT_30_DAY_WINDOW,TERMINAL_ID_TXN_COUNT_30_DAY_WINDOW,TERMINAL_ID_FRAUD_RISK_30_DAY_WINDOW
TRANSACTION_ID,,,,
30686,2018-04-04 07:23:17,0.0,1.0,0.0
34395,2018-04-04 13:04:40,0.0,2.0,0.0
41647,2018-04-05 09:41:47,0.0,3.0,0.0
42835,2018-04-05 11:24:38,0.0,4.0,0.0
43224,2018-04-05 11:57:24,0.0,5.0,0.0
...,...,...,...,...
1725313,2018-09-27 22:05:06,2.0,37.0,0.1
1732564,2018-09-28 15:32:22,1.0,37.0,0.0
1735365,2018-09-29 02:35:48,1.0,38.0,0.0


### Save features


In [34]:
# Saving as parquet makes it easier to pull a single column if needed.
transactions_df.to_parquet('data/simulated/features.parquet')

# Learnings, comments and thoughts
This feature engineering pipeline is ok for now, but in a real life scenario we'd make a function for to engineer each feature and put it into a pipeline.
e.g.
`see pseudo-code below`

You would also do a lot more validation, logging, etc.

In [35]:
def feature_eng_helper(data, feat_name, feat_eng_fn, validation_fn, dependencies, lineage_dict):
    """Helper function to build a single feature and validate the output"""
    # Build feature
    feature = feat_eng_fn(data[dependencies])
    data[feat_name] = feature

    # Validate new feature
    validation_fn(data) # can stop or warn depending on issues found

    # Update linage
    lineage_dict[feat_name] = dependencies

    return data, lineage_dict


def feature_eng_pipeline(data):
    """Function to run the feature engineering pipeline for data passed in"""
    lineage_dict = {} # initalise the lineage dictionary

    data, lineage_dict = feature_eng_helper(data, 'TX_ON_WEEKEND', is_on_weekend, val_is_on_weekend, ['TX_DATETIME'], lineage_dict)
    data, lineage_dict = feature_eng_helper(data, 'TX_AT_NIGHT', is_at_night, val_is_at_night, ['TX_DATETIME'], lineage_dict)

    return data, lineage_dict

Even better we could create a feature engineer object that can contain the feature name, function, validation function and dependencies. Class would also include methods to log, etc.

In [36]:
class FeatureEngineer:
    def __init__(self, feat_name, feat_eng_fn, validation_fn, dependencies):
        self.feat_name = feat_name
        self.feat_eng_fn = feat_eng_fn
        self.validation_fn = validation_fn
        self.dependencies = dependencies

    def build(self, data):
        feature = self.feat_eng_fn(data[self.dependencies])
        data[self.feat_name] = feature
        return data

    def validate(self, data):
        self.validation_fn(data)

In [37]:
# f_weekend = FeatureEngineer(feat_name='TX_ON_WEEKEND',
#                             feat_eng_fn=is_on_weekend,
#                             validation_fn=val_is_on_weekend,
#                             dependencies=['TX_DATETIME'])

In [38]:
# data = f_weekend.build(data)
# f_weekend.validate(data)
# Could create many then loop through sequentially (assuming no parallel is required)